In [1]:
!pip install folium #install folium library for map display

     |████████████████████████████████| 102kB 7.2MB/s ta 0:00:011


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
start_address = 'Badlapur, MH' # start point

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(start_address)
start_latitude = location.latitude
start_longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(start_address,start_latitude, start_longitude))

end_address = 'Airoli, MH' # end point

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(end_address)
end_latitude = location.latitude
end_longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(end_address,end_latitude, end_longitude))

The geograpical coordinate of Badlapur, MH are 19.1561967, 73.2660724.
The geograpical coordinate of Airoli, MH are 19.1582719, 72.9967088.


In [4]:
#get route coordinates between source and destination
import requests

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}

api_key = '<YOUR API KEY>'
web_string = 'https://api.openrouteservice.org/v2/directions/driving-car?api_key={}&start={},{}&end={},{}'.format(
    api_key,
    start_longitude,
    start_latitude,
    end_longitude,
    end_latitude
)

results= requests.get(web_string, headers=headers).json()




In [5]:
#extract the coordinates of route and convert it into data frame

route_coordinates = results['features'][0]['geometry']['coordinates']


route_coordinates_df = pd.DataFrame(route_coordinates)
route_coordinates_df.columns = ['Longitude', 'Latitude']

#route_coordinates_df['Checkpoints'] = np.arange('Check_Point' + str(len(route_coordinates_df)))

route_coordinates_df['Checkpoints'] = np.arange(len(route_coordinates_df)) 
route_coordinates_df['Checkpoints'] = 'CP' + route_coordinates_df['Checkpoints'].astype(str)

fixed_columns = [route_coordinates_df.columns[-1]] + list(route_coordinates_df.columns[:-1])
route_coordinates_df = route_coordinates_df[fixed_columns]


route_coordinates_df.head()










,Checkpoints,Longitude,Latitude
0,CP0,73.266186,19.156360
1,CP1,73.266083,19.156424
2,CP2,73.266000,19.156513
3,CP3,73.265926,19.156563
4,CP4,73.265882,19.156616


In [6]:
LIMIT = 100
CLIENT_ID = '<YOUR CLIENT KEY>' # your Foursquare ID
CLIENT_SECRET = '<YOUR CLIENT SECRET KEY>' # your Foursquare Secret
VERSION = '20200518' # Foursquare API version


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
#radius = 500


    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #print(url)    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)





#get venues nearby for each coordinates
enroute_venues = getNearbyVenues(names=route_coordinates_df['Checkpoints'],
                                   latitudes=route_coordinates_df['Latitude'],
                                   longitudes=route_coordinates_df['Longitude']
                                  )

In [8]:
#enroute_venues['Venue Category'].value_counts()

#enroute_venues.head()
enroute_venues_final = enroute_venues
#drop the Neighborhood cols as we dont require them now
enroute_venues_final.drop('Neighborhood',axis = 1, inplace = True)
enroute_venues_final.drop('Neighborhood Latitude',axis = 1, inplace = True)
enroute_venues_final.drop('Neighborhood Longitude',axis = 1, inplace = True)

enroute_venues_final.head()


,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Radhe Govind Building Material Suppliers,19.156854,73.263643,Grocery Store
1,Radhe Govind Building Material Suppliers,19.156854,73.263643,Grocery Store
2,Radhe Govind Building Material Suppliers,19.156854,73.263643,Grocery Store
3,Radhe Govind Building Material Suppliers,19.156854,73.263643,Grocery Store
4,Radhe Govind Building Material Suppliers,19.156854,73.263643,Grocery Store


In [9]:
enroute_venues_final.drop_duplicates(subset=None,keep = 'first',inplace = True) # drop duplicates
enroute_venues_final.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Radhe Govind Building Material Suppliers,19.156854,73.263643,Grocery Store
21,Aarogya Sadhna Gym,19.165650,73.245180,Athletics & Sports
24,HP World,19.166034,73.243904,Electronics Store
47,Badlapur Rly Station,19.166918,73.239005,Train Station
48,Badlapur,19.166308,73.238994,Metro Station


In [10]:
#get fuel pump data    
    
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_9cfecb853f294f36a37e619be619d809 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='<YOUR API KEY>',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='<YOUR ENDPOINT URL>')

body = client_<NAME>.get_object(Bucket='<YOUR BUCKET KEY>',Key='fuel_pumps.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_fuel_data = pd.read_csv(body)
df_fuel_data.head()


,Venue,Venue Latitude,Venue Longitude,Venue Category
0,surya petrol pump,19.209382,73.185657,Fuel Pump
1,surya petrol pump,19.243006,73.124769,Fuel Pump
2,surya petrol pump,19.267518,73.084461,Fuel Pump
3,surya petrol pump,19.221869,73.017840,Fuel Pump
4,surya petrol pump,19.151184,72.999489,Fuel Pump


In [11]:
enroute_venues_final_appended= enroute_venues_final.append(df_fuel_data)  #append feul pump data to enroute venues


In [12]:
enroute_venues_fina3 =enroute_venues_final_appended #making a copy
enroute_venues_fina3['Venue Category']=enroute_venues_fina3['Venue Category'].str.lower() #converting lower so that it can be compared to filter which will be converted to lower case as well
enroute_venues_fina3.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Radhe Govind Building Material Suppliers,19.156854,73.263643,grocery store
21,Aarogya Sadhna Gym,19.165650,73.245180,athletics & sports
24,HP World,19.166034,73.243904,electronics store
47,Badlapur Rly Station,19.166918,73.239005,train station
48,Badlapur,19.166308,73.238994,metro station


In [13]:
enroute_venues_final2= enroute_venues_fina3 #This is our final dataset with all the places along the route
#enroute_venues_final3= enroute_venues_final

FilterValue="fuel" # Filtering the dataset to check on Fuel pumps
FilterValue=FilterValue.lower()


enroute_venues_final = enroute_venues_final2[enroute_venues_final2['Venue Category'].str.contains(FilterValue)].reset_index(drop=True)
enroute_venues_final.head() #This is our final dataset with all the places along the route with required filter.

#if you want to show all the places without filter user enroute_venues_fina3


#enroute_venues_int['Venue Category'].unique()
#print('There are {} uniques categories.'.format(len(enroute_venues_int['Venue Category'].unique())))

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,surya petrol pump,19.209382,73.185657,fuel pump
1,surya petrol pump,19.243006,73.124769,fuel pump
2,surya petrol pump,19.267518,73.084461,fuel pump
3,surya petrol pump,19.221869,73.017840,fuel pump
4,surya petrol pump,19.151184,72.999489,fuel pump


In [14]:
#get the route coordinates and conver them to list of tuples.
#This will help us plot the line/route on folium map
mydata_df=route_coordinates_df[['Latitude','Longitude']]

mylist = list(mydata_df.itertuples(index=False, name=None))

In [15]:
enroute_venues_final2= enroute_venues_fina3 #This is our final dataset with all the places along the route
#enroute_venues_final3= enroute_venues_final

FilterValue="restaurant" # Filtering the dataset to check on Fuel pumps
FilterValue=FilterValue.lower()


enroute_venues_final_restaurant = enroute_venues_final2[enroute_venues_final2['Venue Category'].str.contains(FilterValue)].reset_index(drop=True)
enroute_venues_final_restaurant.head() #This is our final dataset with all the places along the route with required filter.

#if you want to show all the places without filter user enroute_venues_fina3


#enroute_venues_int['Venue Category'].unique()
#print('There are {} uniques categories.'.format(len(enroute_venues_int['Venue Category'].unique())))

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hotel Aradhana,19.167220,73.238302,indian restaurant
1,Pappu Puri Baji,19.167522,73.237461,indian restaurant
2,Yogeshwar Fine-Dine & Banquet,19.170890,73.227580,restaurant
3,kohinoor Hotel,19.174831,73.232836,indian restaurant
4,Rehan Biryani Corner,19.206441,73.195871,restaurant


In [16]:
# create map for restaurants
map_clusters = folium.Map(location=[start_latitude, start_longitude], zoom_start=11)


# add markers to the map
#markers_colors = []
for lat, lon, poi in zip(enroute_venues_final_restaurant['Venue Latitude'], enroute_venues_final_restaurant['Venue Longitude'], enroute_venues_final_restaurant['Venue']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Marker(
        location =[lat, lon],
        #radius=5,
        popup=label,
        icon=folium.Icon(color='green')
        #color=rainbow[cluster-1],
        #fill=True,
        #fill_color=rainbow[cluster-1],
        #fill_opacity=0.7
    
    ).add_to(map_clusters)
    folium.PolyLine(mylist, weight=2.5, opacity=1).add_to(map_clusters)

    
#label_start = "Start_Point"
#label_end = "End_Point"

folium.Marker(
        location =[start_latitude, start_longitude],
        #radius=5,
        popup=start_address,
        icon=folium.Icon(color='blue')
        #color=rainbow[cluster-1],
        #fill=True,
        #fill_color=rainbow[cluster-1],
        #fill_opacity=0.7
    
    ).add_to(map_clusters)
    
folium.Marker(
        location =[end_latitude, end_longitude],
        #radius=5,
        popup=end_address,
        icon=folium.Icon(color='blue')
        #color=rainbow[cluster-1],
        #fill=True,
        #fill_color=rainbow[cluster-1],
        #fill_opacity=0.7
    
    ).add_to(map_clusters)

map_clusters

In [17]:
# create map for Fuel pump
map_clusters = folium.Map(location=[start_latitude, start_longitude], zoom_start=11)


# add markers to the map
#markers_colors = []
for lat, lon, poi in zip(enroute_venues_final['Venue Latitude'], enroute_venues_final['Venue Longitude'], enroute_venues_final['Venue']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Marker(
        location =[lat, lon],
        #radius=5,
        popup=label,
        icon=folium.Icon(color='green')
        #color=rainbow[cluster-1],
        #fill=True,
        #fill_color=rainbow[cluster-1],
        #fill_opacity=0.7
    
    ).add_to(map_clusters)
    folium.PolyLine(mylist, weight=2.5, opacity=1).add_to(map_clusters)

    

folium.Marker(
        location =[start_latitude, start_longitude],
        #radius=5,
        popup=start_address,
        icon=folium.Icon(color='blue')
        #color=rainbow[cluster-1],
        #fill=True,
        #fill_color=rainbow[cluster-1],
        #fill_opacity=0.7
    
    ).add_to(map_clusters)
    
folium.Marker(
        location =[end_latitude, end_longitude],
        #radius=5,
        popup=end_address,
        icon=folium.Icon(color='blue')
        #color=rainbow[cluster-1],
        #fill=True,
        #fill_color=rainbow[cluster-1],
        #fill_opacity=0.7
    
    ).add_to(map_clusters)

map_clusters